# Lucas Kanade Example

### In this code:

1. We start by capturing video frames from a video file (you can also use a camera).
2. We track feature points using the Lucas-Kanade algorithm in the lucas_kanade_point_tracking function.
3. We draw the tracked points and the motion vectors on each frame.
4. The tracking continues until you press the 'Esc' key to exit the application.

- Make sure to replace 'video.mp4' with the path to your video file or configure the code to use your camera as the video source. 
- You may also need to adjust the parameters for good feature point detection (cv2.goodFeaturesToTrack) to suit your specific tracking needs.

In [1]:
import cv2
import numpy as np


In [2]:

def lucas_kanade_point_tracking(prev_frame, curr_frame, prev_points):
    # Parameters for Lucas-Kanade optical flow
    lk_params = dict(winSize=(15, 15),
                     maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Calculate optical flow using Lucas-Kanade
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_frame, curr_frame, prev_points, None, **lk_params)
    status = status.ravel()
    # Filter out the points with status=1 (successfully tracked)
    prev_points = prev_points[status == 1]
    next_points = next_points[status == 1]

    return prev_points, next_points


In [3]:
# Load video or capture frames from a camera
cap = cv2.VideoCapture('..\\10231.mp4')

# Read the first frame
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Define initial points to track (e.g., use cv2.goodFeaturesToTrack)
prev_points = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.3, minDistance=7)

while True:
    ret, curr_frame = cap.read()
    if not ret:
        break

    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Track points using Lucas-Kanade
    prev_points, next_points = lucas_kanade_point_tracking(prev_gray, curr_gray, prev_points)

    # Draw tracks on the current frame
    for i, (prev, next) in enumerate(zip(prev_points, next_points)):
        x1, y1 = prev.ravel()
        x2, y2 = next.ravel()
        cv2.line(curr_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
        cv2.circle(curr_frame, (int(x2), int(y2)), 5, (0, 255, 0), -1) 
        
    cv2.namedWindow("Optical Flow", cv2.WINDOW_NORMAL)
    cv2.imshow('Optical Flow', curr_frame)
    if cv2.waitKey(0) & 0xFF == 27:  # Press 'Esc' to exit
        break



In [4]:
cap.release()
cv2.destroyAllWindows()
